In [ ]:
import numpy as np
from tensorflow.keras.layers import Embedding, LSTM, Conv2D, MaxPooling2D, Flatten, Dense, Input, concatenate
from tensorflow.keras.models import Model

In [ ]:
# Define the text input
text_input = Input(shape=(100,), name='text_input')
x1 = Embedding(input_dim=10000, output_dim=128)(text_input)
x1 = LSTM(64)(x1)

In [ ]:
# Define the image input
image_input = Input(shape=(64, 64, 3), name='image_input')
x2 = Conv2D(32, (3, 3), activation='relu')(image_input)
x2 = MaxPooling2D((2, 2))(x2)
x2 = Conv2D(64, (3, 3), activation='relu')(x2)
x2 = MaxPooling2D((2, 2))(x2)
x2 = Flatten()(x2)
x2 = Dense(64, activation='relu')(x2)

In [ ]:
# Combine the outputs of the two branches
combined = concatenate([x1, x2])

# Add a final dense layer for classification
output = Dense(1, activation='sigmoid')(combined)

# Create the model
model = Model(inputs=[text_input, image_input], outputs=output)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
text_data = np.random.randint(1, 10000, size=(1000, 100))  # 1000 samples, each of length 100
image_data = np.random.random((1000, 64, 64, 3))  # 1000 samples of 64x64 RGB images
labels = np.random.randint(0, 2, size=(1000, 1))  # Binary labels

In [ ]:
model.fit([text_data, image_data], labels, epochs=10, batch_size=32)